In [26]:
from konlpy.tag import Twitter
from gensim.models import Word2Vec
import csv
from konlpy.tag import Twitter
from konlpy.tag import Hannanum
from konlpy.tag import Kkma
from konlpy.tag import Komoran
from konlpy.tag import Okt
import pandas as pd
import re
from sklearn.feature_extraction.text import CountVectorizer
from sklearn.feature_extraction.text import TfidfTransformer
from sklearn.linear_model import SGDClassifier
from sklearn.pipeline import Pipeline
import tensorflow as tf
import matplotlib

import matplotlib.pyplot as plt
import numpy as np
from sklearn.model_selection import train_test_split

import csv
import logging
from konlpy import jvm
logger = logging.getLogger(__name__)
jvm.init_jvm()



In [27]:
data_set = pd.read_excel("data.xlsx")
data_set = data_set.iloc[:,1:]
data_set.rename({0:'news'},axis=1, inplace= True)
data_label = {1:'culture',2:'global',3:'politic',4:'society',5:'economy'}




In [28]:
for i in range(len(data_set)) :
    data_set['news'][i] = re.sub('[^a-zA-Z0-9 ㄱ-ㅣ가-힣]', '', data_set['news'][i])
    


C:\Users\ICT01_18\Anaconda3\envs\tf_test\lib\site-packages\ipykernel_launcher.py:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  


In [29]:
data_set = data_set.sample(frac=1).reset_index(drop=True)

In [30]:
data_set = data_set.iloc[:40000,:]  # 5만 이상 데이터는 오류 발생(왜?)

In [31]:
data_set.head()

,news,label
0,테슬라 국내 판매 가격 조정최대 5830만원,1
1,이영애 방탄소년단 콘서트 인증샷 공개신곡 기대,1
2,보복조치 에 부메랑유후인벳푸 호텔3곳 1천100명 예약취소,2
3,전국 맑고 일교차 큰 날씨안개미세먼지 주의,1
4,정부 해양조사선 독도 항행에 인정 못 해 강력 항의종합,2


In [32]:
data_set.isna().sum()

news     0
label    0
dtype: int64

In [33]:
def get_noun(text):
    tokenizer = Twitter()
    nouns = tokenizer.nouns(text)
    return [n for n in nouns]

def get_noun2(text):
    tokenizer = Hannanum()
    nouns = tokenizer.nouns(text)
    return [n for n in nouns]

def get_noun3(text):
    tokenizer = Okt()
    nouns = tokenizer.nouns(text)
    return [n for n in nouns]

def get_morphs3(text):
    tokenizer = Okt()
    morphs = tokenizer.morphs(text)
    return [n for n in morphs]

def get_noun4(text):
    tokenizer = Komoran()
    nouns = tokenizer.nouns(text)
    return [n for n in nouns]


In [34]:
train_set, test_set = train_test_split(data_set, test_size = 0.3, random_state = 42)

In [35]:
train_set=train_set.reset_index(drop=True)
test_set=test_set.reset_index(drop=True)

In [36]:
#fit_transform = CV.fit_transform(train_set["news"])

In [37]:
#label_set = data_set['label']
label_set = train_set['label']
len(label_set)

28000

In [38]:
train_set

,news,label
0,베트남 아내 폭행 남성의 티셔츠에 새겨진 말 Stay Humble,4
1,씨름 이 좋은 걸 할배들만 봤네 열광스포츠 예능 시대,1
2,이슈레터 역대 대통령 지지율 반토막 만든 결정적 한방은,3
3,그것이 알고 싶다 캘리포니아 살인 사건 용의자 추적,4
4,강남의 밤을 매수한 01 버닝썬은 픽션이 아니다,1
...,...,...
27995,나몰라라 파업에르노삼성 부산공장 29일부터 셧다운,1
27996,지소미아 종료 1주일앞대통령 원칙론으로 태도변화 압박,3
27997,동성 연인에서 데이트 폭력 피해자로,4
27998,법조계 수사방해 목적으로 장관 인사권 행사땐 직권남용,3


In [39]:
from konlpy.tag import Twitter
from konlpy.tag import Kkma
tf.reset_default_graph()
twitter = Twitter()
token = []
embeddingmodel = []
for i in range(len(data_set)):
    label = data_set['label']
    sentence = twitter.pos(data_set['news'][i], norm=True, stem=True)
    temp = []
    temp_embedding = []
    all_temp = []
    for k in range(len(sentence)):
        temp_embedding.append(sentence[k][0])
        temp.append(sentence[k][0] + '/' + sentence[k][1])
    all_temp.append(temp)
    embeddingmodel.append(temp_embedding)
    
    all_temp.append(label[i])
    token.append(all_temp)
    
embeddingmodel = []
for i in range(len(label_set)):
    temp_embeddingmodel = []
    for k in range(len(token[i][0])):
        temp_embeddingmodel.append(token[i][0][k])
    embeddingmodel.append(temp_embeddingmodel)
embedding  = Word2Vec(embeddingmodel, size = 300, window =5, min_count = 10,
                      iter = 5, sg=1)

C:\Users\ICT01_18\Anaconda3\envs\tf_test\lib\site-packages\konlpy\tag\_okt.py:16: UserWarning: "Twitter" has changed to "Okt" since KoNLPy v0.4.5.
  warn('"Twitter" has changed to "Okt" since KoNLPy v0.4.5.')


In [40]:
embedding.most_similar('코로나/Noun')

C:\Users\ICT01_18\Anaconda3\envs\tf_test\lib\site-packages\ipykernel_launcher.py:1: DeprecationWarning: Call to deprecated `most_similar` (Method will be removed in 4.0.0, use self.wv.most_similar() instead).
  """Entry point for launching an IPython kernel.


[('신종/Noun', 0.9903484582901001),
 ('폐렴/Noun', 0.9758926630020142),
 ('우한/Noun', 0.9711862802505493),
 ('코로나바이러스/Noun', 0.9571097493171692),
 ('국내/Noun', 0.9462285041809082),
 ('확/Noun', 0.9444772005081177),
 ('전세기/Noun', 0.9389653205871582),
 ('번째/Suffix', 0.9301178455352783),
 ('확진/Noun', 0.9293054342269897),
 ('진자/Noun', 0.9289625883102417)]

In [41]:
print(token[0][0])
print(token[1][0])
print(token[2][0])
data_set


['테슬라/Noun', '국내/Noun', '판매/Noun', '가격/Noun', '조정/Noun', '최대/Noun', '5830만원/Number']
['이영애/Noun', '방탄소년단/Noun', '콘서트/Noun', '인증샷/Noun', '공개/Noun', '신곡/Noun', '기대/Noun']
['보복조치/Noun', '에/Josa', '부메랑/Noun', '유후인/Noun', '벳푸/Noun', '호텔/Noun', '3/Number', '곳/Noun', '1천/Number', '100/Number', '명/Noun', '예약/Noun', '취소/Noun']


,news,label
0,테슬라 국내 판매 가격 조정최대 5830만원,1
1,이영애 방탄소년단 콘서트 인증샷 공개신곡 기대,1
2,보복조치 에 부메랑유후인벳푸 호텔3곳 1천100명 예약취소,2
3,전국 맑고 일교차 큰 날씨안개미세먼지 주의,1
4,정부 해양조사선 독도 항행에 인정 못 해 강력 항의종합,2
...,...,...
39995,야생 토끼 먹었다 또 흑사병 확진중국인 불안감,2
39996,엔드게임 신기록 행진4월 영화 점유율 역대 최저,1
39997,구자경 LG 명예회장 별세25년간 글로벌 초우량 LG 도약 이끌,5
39998,깡통전세 불안감 확산 1억짜리가 8000만원대에 낙찰 경매시장에,5


In [42]:
def One_hot(data):
       
    index_dict = {value:index for index,value in enumerate(set(data))}
    result = []

    for value in data:

        one_hot = np.zeros(len(index_dict))
        index = index_dict[value]
        one_hot[index] = 1
        result.append(one_hot)

    return result


def Convert2Vec(model_name, doc): 
    #train_X_ = W2V.Convert2Vec("Word2Vec_csv_article.embedding",train_X)
    word_vec = []
    model = model_name
    for sent in doc:
        sub = []
        for word in sent:
            if word in model.wv.vocab:
                sub.append(model.wv[word]) 
            else:
                sub.append(np.random.uniform(-0.25,0.25,300)) # 사전에 해당 워드가 없으면 랜덤한 vector 생성하게 해줌
        word_vec.append(sub)

    return word_vec





In [43]:
class Bi_LSTM():
    
    def __init__(self, lstm_units, num_class, keep_prob):
        
        self.lstm_units = lstm_units
        
        with tf.variable_scope('forward', reuse = tf.AUTO_REUSE):
            
            self.lstm_fw_cell = tf.nn.rnn_cell.LSTMCell(lstm_units, forget_bias=1.0, state_is_tuple=True)
            self.lstm_fw_cell = tf.contrib.rnn.DropoutWrapper(self.lstm_fw_cell, output_keep_prob = keep_prob)
            
        with tf.variable_scope('backward', reuse = tf.AUTO_REUSE):
            
            self.lstm_bw_cell = tf.nn.rnn_cell.LSTMCell(lstm_units, forget_bias=1.0, state_is_tuple=True)
            self.lstm_bw_cell = tf.contrib.rnn.DropoutWrapper(self.lstm_fw_cell, output_keep_prob = keep_prob)
        
        with tf.variable_scope('Weights', reuse = tf.AUTO_REUSE):
           
            self.W = tf.get_variable(name="W", shape=[2 * lstm_units, num_class],
                                dtype=tf.float32, initializer = tf.contrib.layers.xavier_initializer())
            self.b = tf.get_variable(name="b", shape=[num_class], dtype=tf.float32,
                                initializer=tf.zeros_initializer())
            
            
    def logits(self, X, W, b, seq_len):
        
        (output_fw, output_bw), states = tf.nn.bidirectional_dynamic_rnn(self.lstm_fw_cell, self.lstm_bw_cell,dtype=tf.float32,
                                                                            inputs = X, sequence_length = seq_len)

        outputs = tf.concat([states[0][1], states[1][1]], axis=1)
        pred = tf.matmul(outputs, W) + b        
        return pred
        
    def model_build(self, logits, labels, learning_rate = 0.001):
        
        with tf.variable_scope("loss"):
            
            loss = tf.reduce_mean(tf.nn.softmax_cross_entropy_with_logits_v2(logits = logits , labels = labels)) # Softmax loss
            optimizer = tf.train.AdamOptimizer(learning_rate=learning_rate).minimize(loss) 
            merged = tf.summary.merge_all()
        return loss, optimizer, merged
    
    def graph_build(self, avg_loss, avg_acc):
        
        tf.summary.scalar('Loss', avg_loss)
        tf.summary.scalar('Accuracy', avg_acc)
        merged = tf.summary.merge_all()
        return merged

In [44]:
def Zero_padding(train_batch_X, Batch_size, Maxseq_length, Vector_size):
        
    zero_pad = np.zeros((Batch_size, Maxseq_length, Vector_size))
    for i in range(Batch_size):
        zero_pad[i, :np.shape(train_batch_X[i])[0], :np.shape(train_batch_X[i])[1] ] = train_batch_X[i]
    return zero_pad

In [45]:
tokens = np.array(token)
data_x = tokens[:,0]
data_y = tokens[:,1]


data_y = One_hot(data_y)
data_x = Convert2Vec(embedding, data_x)

train_x, test_x, train_y, test_y = train_test_split(data_x,data_y,
                                                    test_size = 0.3,
                                                    random_state = 42)
print(len(train_x))
print(len(train_y))
print(len(test_x))
#train_x = train_x.reset_index(drop=True)
#train_y = train_y.reset_index(drop=True)
#test_x = test_x.reset_index(drop=True)
#test_y = test_y.reset_index(drop=True)

28000
28000
12000


In [46]:
len(data_x[0][0])

300

In [47]:
tf.reset_default_graph()
Batch_size = 32
Total_size = len(train_x)
Vector_size = 300
seq_length = [len(x) for x in train_x]
Maxseq_length = max(seq_length)
learning_rate = 0.001
lstm_units = 128
num_class = 5
training_epochs = 5
keep_prob = 0.75

X = tf.placeholder(tf.float32, shape = [None, Maxseq_length, Vector_size])
Y = tf.placeholder(tf.float32, shape = [None, num_class])
seq_len = tf.placeholder(tf.int32 , shape = [None])

BiLSTM = Bi_LSTM(lstm_units, num_class, keep_prob)
with tf.variable_scope('loss', reuse = tf.AUTO_REUSE) :
    logits = BiLSTM.logits(X, BiLSTM.W, BiLSTM.b, seq_len)
    loss, optimizer, merged = BiLSTM.model_build(logits, Y, learning_rate)

prediction = tf.nn.softmax(logits)
correct_pred = tf.equal(tf.argmax(prediction, 1), tf.argmax(Y, 1))
accuracy = tf.reduce_mean(tf.cast(correct_pred, tf.float32))

init = tf.global_variables_initializer()

total_batch = int(Total_size / Batch_size)

print("Start training!")

#modelName = "BiLSTM.model"
saver = tf.train.Saver()

Start training!


In [48]:
test_set

,news,label
0,제 발등 찍은 일본대일 무역적자 16년만에 최저치 전망,5
1,잠적 북한외교관 딸 북송 이탈리아 정가에 파문종합2보,2
2,정권 애완견 하다 의원 되는 게 평범한 정의냐 진중권 여당행,3
3,자식 넷 모두 명문 의대 보낸 거실 공부가 비결,4
4,블루보틀 상륙에 울고 웃는 사람들,5
...,...,...
11995,날씨 포근한 초봄 날씨에 미세먼지 내일 출근길 비눈,1
11996,단독 윤석열 대학살 충격측근들 불러 해야할 일 했다,4
11997,속보 문 대통령 조국 사태 후 윤석열 총장과 첫 대면윤 총장 거,3
11998,WHO 중국에 국제 전문가 파견 합의시진핑 악마와 싸움 이길,2


In [49]:
print(test_set['news'])
tokens[2][0]

0             제 발등 찍은 일본대일 무역적자 16년만에 최저치 전망
1              잠적 북한외교관 딸 북송 이탈리아 정가에 파문종합2보
2         정권 애완견 하다 의원 되는 게 평범한 정의냐 진중권 여당행 
3                 자식 넷 모두 명문 의대 보낸 거실 공부가 비결
4                         블루보틀 상륙에 울고 웃는 사람들
                        ...                 
11995           날씨 포근한 초봄 날씨에 미세먼지 내일 출근길 비눈
11996           단독 윤석열 대학살 충격측근들 불러 해야할 일 했다
11997    속보 문 대통령 조국 사태 후 윤석열 총장과 첫 대면윤 총장 거
11998      WHO 중국에 국제 전문가 파견 합의시진핑 악마와 싸움 이길
11999           밀착마크김부겸 새마을 척결이 훈장 이게 진보 넌센스
Name: news, Length: 12000, dtype: object


['보복조치/Noun',
 '에/Josa',
 '부메랑/Noun',
 '유후인/Noun',
 '벳푸/Noun',
 '호텔/Noun',
 '3/Number',
 '곳/Noun',
 '1천/Number',
 '100/Number',
 '명/Noun',
 '예약/Noun',
 '취소/Noun']

In [50]:
import time
import os
result  = []
loss_graph = []
loss_graph_test=[]
with tf.Session() as sess:
    #writer = tf.summary.FileWriter('', sess.graph)
    start_time = time.time()
    sess.run(init)
    train_writer = tf.summary.FileWriter('Bidirectional_LSTM', sess.graph)
    i = 0
    for epoch in range(training_epochs):
        #try :
            avg_acc, avg_loss = 0. , 0.
            for step in range(total_batch):

                train_batch_X = train_x[step*Batch_size : step*Batch_size+Batch_size]

                train_batch_Y = train_y[step*Batch_size : step*Batch_size+Batch_size]
                batch_seq_length = seq_length[step*Batch_size : step*Batch_size+Batch_size]
                train_batch_X = Zero_padding(train_batch_X, Batch_size, Maxseq_length, Vector_size)
                
                #summary, _ = sess.run([merged,optimizer], feed_dict={X: train_batch_X, Y: train_batch_Y, seq_len: batch_seq_length})

                sess.run(optimizer, feed_dict={X: train_batch_X, Y: train_batch_Y, seq_len: batch_seq_length})
                
                loss_ = sess.run(loss, feed_dict={X: train_batch_X, Y: train_batch_Y, seq_len: batch_seq_length})
                avg_loss += loss_ / total_batch

                acc = sess.run(accuracy , feed_dict={X: train_batch_X, Y: train_batch_Y, seq_len: batch_seq_length})
                avg_acc += acc / total_batch
                #train_writer.add_summary(summary, step)
            print("epoch : {:02d} step : {:04d} loss = {:.6f} accuracy= {:.6f}".format(epoch+1, step+1, loss_, acc))
            #print("Test Accuracy : ", sess.run(accuracy, feed_dict={X:test_x, Y:test_y}))
            #summary = sess.run(BiLSTM.graph_build(avg_loss, avg_acc))       
            #train_writer.add_summary(summary, i)
            #merged = BiLSTM.graph_build()
            i += 1
        #except : 
            #print('Error Epoch : ', epoch)
            loss_graph.append([acc,epoch])
            #acc = sess.run(accuracy , feed_dict={X: test_x, Y: test_y, seq_len: batch_seq_length})
            #print('Test Accuracy : ', acc)

    duration = time.time() - start_time
    minute = int(duration / 60)
    second = int(duration) % 60
    print("%dminutes %dseconds" % (minute,second))
    save_path = saver.save(sess, os.getcwd())

    train_writer.close()
    print('save_path',save_path)
    
    test_size = len(test_x)
    test_batch = int(test_size / Batch_size)
    seq_length = [len(x) for x in test_x]
    keep_prob = 1.0
    total_acc = 0
    for step in range(test_batch):
        test_batch_X = test_x[step*Batch_size: step*Batch_size + Batch_size]
        test_batch_Y = test_y[step*Batch_size: step*Batch_size + Batch_size]
        batch_seq_length = seq_length[step*Batch_size : step*Batch_size + Batch_size]
        test_batch_X = Zero_padding(test_batch_X, Batch_size, Maxseq_length, Vector_size)
        
        #summary, _ = sess.run([mreged, optimizer], feed_dict = {X: test_batch_X, Y: test_batch_Y, seq_len: batch_seq_length})
        acc = sess.run(accuracy , feed_dict={X: test_batch_X, Y: test_batch_Y, seq_len: batch_seq_length})
        total_acc += acc/test_batch
        pred_v = sess.run(prediction , feed_dict={X: test_batch_X, Y: test_batch_Y, seq_len: batch_seq_length})
 
        print("loss = {:.6f} accuracy = {:.6f}".format(loss_, acc), "/ step =",step)
            
            
    print(" Total Accuracy : {}".format(total_acc))
    result.append(total_acc)

    
print("----------------------------------------")
print("BI-LSTM finish")
for i in range(0, len(result)):
    print("epoch : ", i+1, " / accuracy : ", result[i])
    
print("\nBest Result epoch : ", result.index(max(result)) + 1, " / accuracy: ", max(result))

IndexError: tuple index out of range